<a href="https://colab.research.google.com/github/pyreyes/style_transfer/blob/main/make_vocabularies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import random
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from collections import Counter

from google.colab import drive
drive.mount('/content/drive')

os.chdir('drive/MyDrive/CS6120/Project')

Mounted at /content/drive


In [ ]:
pip install datasets

     |████████████████████████████████| 194kB 8.1MB/s 
     |████████████████████████████████| 245kB 9.6MB/s 
     |████████████████████████████████| 112kB 13.9MB/s 


In [ ]:
from datasets import load_dataset  
dataset = load_dataset("amazon_polarity")

Dataset amazon_polarity downloaded and prepared to /root/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/35b3f5f426863b55dda2d5dc0da2d3714c7b0e5ba64bc4f1415ac9291ffdb234. Subsequent calls will reuse this data.


In [ ]:
import re

def get_review(r):
  sens = re.split('\. |\n', r)
  # re.sub(r'[^A-Za-z ,\' -!&_]+',' ',
  sens = [re.sub(r'[\d]+','_num_',re.sub(r'([^A-Za-z_\d\s\'-])',r' \1 ',sen.lower())) for sen in sens if len(sen) in range(20,160)]
  rand = random.randint(0, max(len(sens)-1,0))
  if len(sens) > 0:
    return sens[rand]
  else:
    return ''


In [ ]:
train = [get_review(r) for r in dataset['train']['content']]

labels = dataset['train']['label']
train_1 = []
train_0 = []

for i, l in enumerate(labels):
  if len(train[i]) < 1:
    continue
  if l == 1:
    train_1.append(train[i])
  else:
    train_0.append(train[i])

In [ ]:
random.shuffle(train_1)
random.shuffle(train_0)
dev_1 = train_1[:340000]
dev_0 = train_0[:340000]
train_1 = train_1[340000:]
train_0 = train_0[340000:]

In [ ]:
#write negative reviews out to file
f = open("amazon/sentiment.train.0", "w")
for l in train_0:
  if len(l) > 0:
    f.write(l+'\n')
    f.close

#write positive reviews out to file
f = open("amazon/sentiment.train.1", "w")
for l in train_1:
  if len(l) > 0:
    f.write(l+'\n')
    f.close

#write positive reviews out to file
f = open("amazon/sentiment.dev.1", "w")
for l in dev_1:
  if len(l) > 0:
    f.write(l+'\n')
    f.close

#write positive reviews out to file
f = open("amazon/sentiment.dev.0", "w")
for l in dev_0:
  if len(l) > 0:
    f.write(l+'\n')
    f.close

In [ ]:
train = train_0 + train_1

corpus = [
          re.sub(r'[^\w\'-]','',w) for l in train
          for w in l.strip().split() 
          if len(re.sub(r'[^\w\'-]','',w)) > 0 
          ]

In [ ]:
k = 20000
vocab = []

vocab.append('<unk>')
vocab.append('<pad>')
vocab.append('<s>')
vocab.append('</s>')

c = Counter()
for l in corpus:
    for tok in l.strip().split():
        c[tok] += 1

for tok, _ in c.most_common(k):
    vocab.append(tok)

In [ ]:
#save vocab to file
f = open("amazon/vocab", "w")
for w in vocab:
  f.write(w+'\n')
  f.close

In [ ]:
#load positive sentiment data
pos_corpus = [
    w #if w in vocab else '<unk>' 
    for l in train_1 #open('amazon/sentiment.train.1')
    for w in l.strip().split()
]

#load negative sentiment data
neg_corpus = [
    w #if w in vocab else '<unk>' 
    for l in train_0 #open('amazon/sentiment.train.0')
    for w in l.strip().split()
]

#define object to calculate the n-gram salience as defined in paper#1
class SalienceCalculator(object):
    def __init__(self, pos_corpus, neg_corpus):
        self.vectorizer = CountVectorizer()

        pos_count_matrix = self.vectorizer.fit_transform(pos_corpus)
        self.pos_vocab = self.vectorizer.vocabulary_
        self.pos_counts = np.sum(pos_count_matrix, axis=0)
        self.pos_counts = np.squeeze(np.asarray(self.pos_counts))

        neg_count_matrix = self.vectorizer.fit_transform(neg_corpus)
        self.neg_vocab = self.vectorizer.vocabulary_
        self.neg_counts = np.sum(neg_count_matrix, axis=0)
        self.neg_counts = np.squeeze(np.asarray(self.neg_counts))


    def salience(self, feature, attribute='neg', lmbda=0.5):
        assert attribute in ['pos', 'neg']

        if feature not in self.pos_vocab:
            pos_count = 0.0
        else:
            pos_count = self.pos_counts[self.pos_vocab[feature]]

        if feature not in self.neg_vocab:
            neg_count = 0.0
        else:
            neg_count = self.neg_counts[self.neg_vocab[feature]]
        
        if attribute == 'pos':
            return (pos_count + lmbda) / (neg_count + lmbda)
        else:
            return (neg_count + lmbda) / (pos_count + lmbda)

#calculate n-gram salience
sc = SalienceCalculator(pos_corpus, neg_corpus)

In [ ]:
# build attribute vocabulary
r = 4
attribute_vocab = []
for tok in vocab:
  if max(sc.salience(tok, attribute='neg'), sc.salience(tok, attribute='pos')) > r:
    attribute_vocab.append(tok)

attribute_vocab = [w for w in attribute_vocab if w in vocab]

In [ ]:
#save attribute_vocab
f = open("amazon/attribute_vocab", "w")
for w in attribute_vocab:
  f.write(w+'\n')
  f.close


In [ ]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import conlltags2tree, tree2conlltags
f = open('amazon/sentiment.train.0', 'r')

for line in f:
	lines=line.strip().split('\t')
	#if(len(lines)!=2 or 'play' in line):
	if(len(lines)!=2):
		continue
	if(string.atof(lines[1])<5.0):
		break
	ne_tree = ne_chunk(pos_tag(word_tokenize(lines[0])))
	iob_tagged = tree2conlltags(ne_tree)
	#print iob_tagged
	flag=0
	for ll in iob_tagged:
		if('NN' in ll[1]):
			flag=1
			break
	if(flag==0):
		fw.write(line)	

In [ ]:
type(test)

NameError: ignored

In [ ]:
#Used to extract data
# import tarfile
# tar = tarfile.open("Amazon Reviews Data/data.tar")
# tar.extractall()
# tar.close()

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Computers.json.gz

--2021-03-29 23:12:05--  http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Computers.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7010521 (6.7M) [application/octet-stream]
Saving to: ‘meta_Computers.json.gz’

meta_Computers.json 100%[===================>]   6.69M  20.0MB/s    in 0.3s    

2021-03-29 23:12:05 (20.0 MB/s) - ‘meta_Computers.json.gz’ saved [7010521/7010521]



In [ ]:
### load the meta data

data = []
with gzip.open('meta_Computers.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[2])

18772
{'description': ['School To Home Notes: Note From the Teacher!\\nSuitable Wording for Both Parents and Caregivers!\\nA great, time-saving way to communicate with parents/care providers! Each specifically designed School to Home Notes booklet contains 60, colorful, two-part c'], 'title': 'TEACHERS FRIEND NOTE FROM THE TEACHER SCHOOL TO HOME NOTES', 'image': ['https://images-na.ssl-images-amazon.com/images/I/31GI7SdbS7L._SS40_.jpg'], 'brand': 'TEACHERS FRIEND', 'rank': ['>#1,233,887 in Electronics (See Top 100 in Electronics)'], 'main_cat': 'Computers', 'date': 'June 6, 2007', 'asin': '0439651859'}


In [ ]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

18772


In [ ]:
### remove rows with unformatted title (i.e. some 'title' may still contain html style content)

df3 = df.fillna('')
df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
print(len(df4))
print(len(df5))

135
18637


In [ ]:
# how those unformatted rows look like
df4.iloc[0]

description                                                      
title           var aPageStart = (new Date()).getTime();\nvar ...
image                                                            
brand                                              DS Miller Inc.
rank            [>#1,826,312 in Electronics (See Top 100 in El...
main_cat                                                Computers
date                                                March 1, 2010
asin                                                   B0016C5EXY
feature                                                          
tech1                                                            
also_buy                                                         
price                                                            
also_view                                                        
tech2                                                            
details                                                          
similar_it

In [ ]:
import sys
import random
import string

data_prefix=data/amazon/sentiment.train.
result_prefix=sentiment.train.
data_file_num=2
negtive_array=[]
def load_data(file_name):
	f=open(file_name,'r')
	tmp=[]
	for line in f:
		if('url' in line):
			continue
		line=line.strip()
		tmp.append(line)
	random.shuffle(tmp)
	return tmp[:min(len(tmp),5000000000)]
#negtive_array+=load_data('all_subtitle.gbk')
#negtive_array+=load_data('all_daily_dialogtwitter_sen.txt')
name_array=[]
for i in range(0,2):
	name_array.append(data/amazon/sentiment.train.+str(i))
#name_array=['funny_train.txt.test','romantic_train.txt.test']
#tag='Sheldon'
def get_dict(sen_array):
        tmp_dict={}
        for i in sen_array:
                sens=i.strip().split(' ')
                for n in range(1,5):
                        for l in range(0,len(sens)-n+1):
                                tmp=' '.join(sens[l:l+n])
                                if(tmp_dict.get(tmp)!=None):
                                        tmp_dict[tmp]+=1
                                else:
                                        tmp_dict[tmp]=1
        return tmp_dict
num=0
for tag in name_array:
	negtive_array=[]
	positve_array=load_data(tag)
	for i in name_array:
		if tag!=i:
			negtive_array+=load_data(i)
	#fw=open(tag+'.train','w')
	#neg_dict={}
	#pos_dict={}
	neg_dict=get_dict(negtive_array)
	pos_dict=get_dict(positve_array)
	tf_idf={}
	for i in pos_dict.keys():
        	if(neg_dict.get(i)!=None):
                	tf_idf[i]=(pos_dict[i]+1.0)/(neg_dict[i]+1.0)
        	else:
                	tf_idf[i]=(pos_dict[i]+1.0)
	tf_dif1=sorted(tf_idf.items(), lambda x, y: cmp(x[1], y[1]), reverse=True)
	fw=open(sentiment.train.+str(num)+'.tf_idf'+'.'+sys.argv[3],'w')
	for i in tf_dif1:
        	fw.write(i[0]+'\t'+str(i[1])+'\n')
	num+=1